In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import metrics
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import scipy

plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False 

In [2]:
data1 = pd.read_excel('used_car_train data2.xls', encoding='gbk').set_index(["SaleID", "name"])
data1

,,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,name,,,,,,,,,,,,,,,,,,,,,
0,736,20040402,30,6,1.0,0.0,0.0,60,12.5,0,1046,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,2262,20030301,40,1,2.0,0.0,0.0,0,15.0,-,4366,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,14874,20040403,115,15,1.0,0.0,0.0,163,12.5,0,2806,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,71865,19960908,109,10,0.0,0.0,1.0,193,15.0,0,434,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,111080,20120103,110,5,1.0,0.0,0.0,68,5.0,0,6977,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2996,189536,20071110,1,1,2.0,NaN,0.0,140,15.0,0,1343,...,0.261753,0.000352,0.138976,0.085219,0.025476,2.106361,-2.359151,2.165224,-0.846520,1.306796
2997,63470,20020204,145,4,5.0,0.0,1.0,343,15.0,0,5751,...,0.283772,0.134105,0.104052,0.061334,0.002554,-7.161608,0.333974,2.171960,-2.240909,-1.597441
2998,5626,20000607,26,14,2.0,0.0,0.0,101,15.0,0,1463,...,0.253778,0.090059,0.056603,0.024048,0.069122,-2.377253,1.792449,-2.847012,0.154435,0.118546


In [3]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3001 entries, (0, 736) to (3000, 59888)
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   regDate            3001 non-null   int64  
 1   model              3001 non-null   int64  
 2   brand              3001 non-null   int64  
 3   bodyType           2922 non-null   float64
 4   fuelType           2827 non-null   float64
 5   gearbox            2876 non-null   float64
 6   power              3001 non-null   int64  
 7   kilometer          3001 non-null   float64
 8   notRepairedDamage  3001 non-null   object 
 9   regionCode         3001 non-null   int64  
 10  seller             3001 non-null   int64  
 11  offerType          3001 non-null   int64  
 12  creatDate          3001 non-null   int64  
 13  price              3001 non-null   int64  
 14  v_0                3001 non-null   float64
 15  v_1                3001 non-null   float64
 16  v_2     

In [4]:
data1.dropna(axis=0,how='all')
data1 = data1[~data1['bodyType'].isin(["NAN"])]
data1 = data1[~data1['fuelType'].isin(["NAN"])]
data1 = data1[~data1['gearbox'].isin(["NAN"])]
data1 = data1[~data1['notRepairedDamage'].isin(["-"])]
data1 = data1.astype('float')
data1

,,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
SaleID,name,,,,,,,,,,,,,,,,,,,,,
0,736,20040402.0,30.0,6.0,1.0,0.0,0.0,60.0,12.5,0.0,1046.0,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
2,14874,20040403.0,115.0,15.0,1.0,0.0,0.0,163.0,12.5,0.0,2806.0,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,71865,19960908.0,109.0,10.0,0.0,0.0,1.0,193.0,15.0,0.0,434.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,111080,20120103.0,110.0,5.0,1.0,0.0,0.0,68.0,5.0,0.0,6977.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482
5,137642,20090602.0,24.0,10.0,0.0,1.0,0.0,109.0,10.0,0.0,3690.0,...,0.260246,0.000518,0.119838,0.090922,0.048769,1.885526,-2.721943,2.457660,-0.286973,0.206573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1674,19991108.0,0.0,0.0,0.0,0.0,0.0,150.0,15.0,0.0,1423.0,...,0.267034,0.108669,0.084197,0.034922,0.031093,-4.240902,1.423684,-1.069548,-1.080304,0.330602
2997,63470,20020204.0,145.0,4.0,5.0,0.0,1.0,343.0,15.0,0.0,5751.0,...,0.283772,0.134105,0.104052,0.061334,0.002554,-7.161608,0.333974,2.171960,-2.240909,-1.597441
2998,5626,20000607.0,26.0,14.0,2.0,0.0,0.0,101.0,15.0,0.0,1463.0,...,0.253778,0.090059,0.056603,0.024048,0.069122,-2.377253,1.792449,-2.847012,0.154435,0.118546


In [5]:
y = data1.iloc[:,13]
x = data1.drop(["price"], axis=1)

In [6]:
# 数据分割 训练集 和 测试集
kf = KFold(n_splits = 10)
dict_datas = {}
count = 0
for train,valid in kf.split(x):
    train_x = np.array(x)[train]
    valid_x = np.array(x)[valid]
    train_x = pd.DataFrame(train_x,columns=list(data1.columns.delete(list(data1.columns).index("price"))))
    valid_x = pd.DataFrame(valid_x,columns=list(data1.columns.delete(list(data1.columns).index("price"))))
    train_y = np.array(y)[train]
    valid_y = np.array(y)[valid]
    train_y = pd.DataFrame(train_y,columns=["price"])
    valid_y = pd.DataFrame(valid_y,columns=["price"])
    dict_datas[count] = (train_x, valid_x,train_y, valid_y)
    count += 1

In [7]:
x_model = sm.add_constant(x)
model = sm.OLS(y,x_model)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     269.6
Date:                Mon, 13 Jun 2022   Prob (F-statistic):               0.00
Time:                        00:06:53   Log-Likelihood:                -22876.
No. Observations:                2347   AIC:                         4.580e+04
Df Residuals:                    2321   BIC:                         4.595e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const               3.78e+07   4.94e+07      0.765      0.444   -5.91e+07    1.35e+08
regDate              -0.0091      0.005     -2.011      0.044      -0.018      -0.000
model                 8.6569      2.400      3.607      0.000       3.951      13.363
brand               -22.4413     12.420     -1.807      0.071     -46.797       1.915
bodyType            218.4694     61.418      3.557      0.000      98.030     338.909
fuelType             53.4592    183.376      0.292      0.771    -306.138     413.057
gearbox            1037.9008    229.016      4.532      0.000     588.804    1486.998
power                19.6541      1.603     12.264      0.000      16.512      22.797
kilometer          -402.5800     29.558    -13.620      0.000    -460.543    -344.617
notRepairedDamage   142.1456    293.671      0.484      0.628    -433.739     718.031
regionCode            0.0684      0.049      1.385      0.166      -0.028       0.165
seller               -0.0004      0.001     -0.698      0.485      -0.002       0.001
offerType          -2.97e-05   2.35e-05     -1.263      0.207   -7.58e-05    1.64e-05
creatDate             2.8963      2.631      1.101      0.271      -2.263       8.056
v_0               -2.255e+06   1.22e+06     -1.852      0.064   -4.64e+06    1.33e+05
v_1                2.495e+07    3.6e+07      0.693      0.488   -4.57e+07    9.56e+07
v_2                2.556e+07   2.48e+07      1.031      0.303    -2.3e+07    7.42e+07
v_3               -3.977e+06   3.43e+06     -1.159      0.246   -1.07e+07    2.75e+06
v_4                1.016e+05   1.58e+05      0.642      0.521   -2.09e+05    4.12e+05
v_5                1.154e+07   1.38e+07      0.835      0.404   -1.56e+07    3.86e+07
v_6                6.048e+06   1.13e+06      5.341      0.000    3.83e+06    8.27e+06
v_7                2.057e+06   2.39e+06      0.860      0.390   -2.63e+06    6.74e+06
v_8                4.051e+06   3.71e+06      1.093      0.274   -3.22e+06    1.13e+07
v_9                3.355e+06   3.47e+06      0.968      0.333   -3.44e+06    1.02e+07
v_10               1.669e+07   2.74e+07      0.610      0.542    -3.7e+07    7.03e+07
v_11               -2.86e+07   3.18e+07     -0.898      0.369   -9.11e+07    3.38e+07
v_12              -1.424e+07   1.27e+07     -1.120      0.263   -3.92e+07    1.07e+07
v_13              -2.618e+05   3.01e+05     -0.868      0.385   -8.53e+05    3.29e+05
v_14               1995.8371   6978.485      0.286      0.775   -1.17e+04    1.57e+04
==============================================================================
Omnibus:                     1480.764   Durbin-Watson:                   1.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39784.241
Skew:                           2.523   Prob(JB):                         0.00
Kurtosis:      

In [8]:
#拟合优度
results.rsquared_adj

0.7410883833648239

In [9]:
#mses = []
maes = []
xs = []
results_list = []
for i in dict_datas:
    
    
    x_model = sm.add_constant(dict_datas[i][0])
    model = sm.OLS(dict_datas[i][2],x_model)
    results = model.fit()
    results_list.append(results)
    xs.append(results.params)
    
    temps = []
    for j in range(len(dict_datas[i][1])):
        temp = np.dot(np.array(results.params[1:]),np.array(dict_datas[i][1].T[j]))+np.array(results.params)[0]
        temps.append(temp)
    temps = np.array(temps)
    MAE = np.mean(np.abs(np.array(dict_datas[i][3].T)[0]-temps))
    maes.append(MAE)
    #MSE = metrics.mean_squared_error(np.array(dict_datas[i][3].T)[0], temps)
    #mses.append(MSE)

In [10]:
min(maes)

2426.476808578949

In [11]:
results_list[maes.index(min(maes))].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.744
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     242.3
Date:                Mon, 13 Jun 2022   Prob (F-statistic):               0.00
Time:                        00:06:55   Log-Likelihood:                -20629.
No. Observations:                2112   AIC:                         4.131e+04
Df Residuals:                    2086   BIC:                         4.146e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              6.137e+07   5.33e+07      1.150      0.250   -4.32e+07    1.66e+08
regDate              -0.0103      0.005     -2.072      0.038      -0.020      -0.001
model                 9.2487      2.569      3.600      0.000       4.210      14.287
brand               -26.0298     13.341     -1.951      0.051     -52.193       0.133
bodyType            214.6069     65.931      3.255      0.001      85.310     343.904
fuelType             23.7296    199.657      0.119      0.905    -367.818     415.277
gearbox            1140.3610    247.097      4.615      0.000     655.778    1624.944
power                19.8542      1.695     11.715      0.000      16.531      23.178
kilometer          -403.4357     31.792    -12.690      0.000    -465.784    -341.088
notRepairedDamage   214.2909    315.573      0.679      0.497    -404.581     833.162
regionCode            0.0668      0.053      1.257      0.209      -0.037       0.171
seller               -0.0005      0.000     -1.463      0.144      -0.001       0.000
offerType             0.0002      0.000      1.149      0.251      -0.000       0.001
creatDate             2.8907      2.844      1.016      0.310      -2.687       8.469
v_0               -2.788e+06   1.31e+06     -2.126      0.034   -5.36e+06   -2.16e+05
v_1                4.174e+07   3.89e+07      1.073      0.283   -3.45e+07    1.18e+08
v_2                3.819e+07   2.67e+07      1.428      0.153   -1.43e+07    9.06e+07
v_3               -5.744e+06    3.7e+06     -1.553      0.121    -1.3e+07    1.51e+06
v_4                1.739e+05   1.71e+05      1.019      0.308   -1.61e+05    5.09e+05
v_5                1.261e+07   1.48e+07      0.850      0.396   -1.65e+07    4.17e+07
v_6                5.957e+06   1.22e+06      4.892      0.000    3.57e+06    8.35e+06
v_7                2.237e+06   2.57e+06      0.871      0.384    -2.8e+06    7.27e+06
v_8                4.327e+06   3.98e+06      1.088      0.277   -3.47e+06    1.21e+07
v_9                3.637e+06   3.72e+06      0.977      0.329   -3.66e+06    1.09e+07
v_10               2.907e+07   2.96e+07      0.984      0.325   -2.89e+07     8.7e+07
v_11              -4.437e+07   3.44e+07     -1.291      0.197   -1.12e+08     2.3e+07
v_12               -2.08e+07   1.37e+07     -1.517      0.129   -4.77e+07    6.09e+06
v_13              -4.049e+05   3.25e+05     -1.245      0.213   -1.04e+06    2.33e+05
v_14               4725.0094   7532.211      0.627      0.531      -1e+04    1.95e+04
==============================================================================
Omnibus:                     1291.783   Durbin-Watson:                   1.967
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32395.556
Skew:                           2.426   Prob(JB):                         0.00
Kurtosis:      

In [12]:
#拟合优度
results_list[maes.index(min(maes))].rsquared_adj

0.740776008432795